# Estimating Rotational Motions for Source Tracking

   <img src="pictures/Merapi_Net2001.png" alt="Merapi Array" width="600"/>

In [ ]:
# Import 
%matplotlib inline

import os
import subprocess
import optparse
import matplotlib
#matplotlib.use("agg")
from obspy import *
from obspy.core import AttribDict
import obspy.signal.array_analysis as AA
import obspy.signal.util as util
import matplotlib.pyplot as plt
from matplotlib.colorbar import ColorbarBase
from matplotlib.colors import Normalize
import matplotlib.cm as cm
import numpy as np
import time
from obspy.signal.rotate import rotate2zne
from obspy.geodetics import gps2dist_azimuth
from obspy.clients.filesystem import sds
import scipy as sp
import scipy.odr as odr
import math


# Computation of "Array-Derived-Rotation" (ADR) using the $L_2$ norm algorithm of Spudich and Fletcher (2008)

## array_rotation_strain(subarray, ts1, ts2, ts3, vp, vs, array_coords, sigmau)

### vp (float)
P wave speed in the soil under the array (km/s)

### vs (float)
S wave speed in the soil under the array Note - vp and vs may be any unit (e.g. miles/week), and this unit need not be related to the units of the station coordinates or ground motions, but the units of vp and vs must be the SAME because only their ratio is used.

### array_coords (numpy.ndarray)
array of dimension na x 3, where na is the number of stations in the array. array_coords[i,j], i in arange(na), j in arange(3) is j coordinate of station i. units of array_coords may be anything, but see the “Discussion of input and output units” above. The origin of coordinates is arbitrary and does not affect the calculated strains and rotations. Stations may be entered in any order.

### ts1 (numpy.ndarray)
array of x1-component seismograms, dimension nt x na. ts1[j,k], j in arange(nt), k in arange(na) contains the k’th time sample of the x1 component ground motion at station k. NOTE that the seismogram in column k must correspond to the station whose coordinates are in row k of in.array_coords. nt is the number of time samples in the seismograms. Seismograms may be displacement, velocity, acceleration, jerk, etc. See the “Discussion of input and output units” below.

### ts2 (numpy.ndarray)
same as ts1, but for the x2 component of motion.

### ts3 (numpy.ndarray)
same as ts1, but for the x3 (UP or DOWN) component of motion.

### sigmau (float or numpy.ndarray)
standard deviation (NOT VARIANCE) of ground noise, corresponds to sigma-sub-u in S95 lines above eqn (A5). NOTE: This may be entered as a scalar, vector, or matrix!

If sigmau is a scalar, it will be used for all components of all stations.
If sigmau is a 1D array of length na, sigmau[i] will be the noise assigned to all components of the station corresponding to array_coords[i,:]
If sigmau is a 2D array of dimension na x 3, then sigmau[i,j] is used as the noise of station i, component j.
In all cases, this routine assumes that the noise covariance between different stations and/or components is zero.

### subarray (numpy.ndarray)
NumPy array of subarray stations to use. I.e. if subarray = array([1, 4, 10]), then only rows 1, 4, and 10 of array_coords will be used, and only ground motion time series in the first, fourth, and tenth columns of ts1 will be used. Nplus1 is the number of elements in the subarray vector, and N is set to Nplus1 - 1. To use all stations in the array, set in.subarray = arange(na), where na is the total number of stations in the array (equal to the number of rows of in.array_coords. Sequence of stations in the subarray vector is unimportant; i.e. subarray = array([1, 4, 10]) will yield essentially the same rotations and strains as subarray = array([10, 4, 1]). “Essentially” because permuting subarray sequence changes the d vector, yielding a slightly different numerical result.

### Caution: Running the algorithm several times will cause overlaps as the additonal runs are appended to the existing files
Solution: delete the files first!

In [ ]:
def getParameters(path2store,t1,t2,array,sub_array):
    par = AttribDict()
    par.t1 = t1
    par.t2 = t2
    par.path2store = path2store
    par.freqmin = 0.01
    par.client = sds.Client(sds_root= "./data_sds/")
    par.stationxmldir = "./stationxml/"
    par.array_stations = array
    par.sub_array = sub_array
    return par
    

In [ ]:
def run(par):
    start = par.t1
    end = par.t2
    print(start)

# Data Base
    localclient = par.client
    array_stations = par.array_stations
    print(localclient)
    print(array_stations)

    subarray = par.sub_array

    res = []
    tsz = []
    tsn = []
    tse = []
    coo = []
    first = True
    for station in array_stations:
        net,sta,stream = station.split(".")
        stats = localclient.get_waveforms(debug=True,network=net,station=sta,location="",\
                                          channel=stream, starttime=start, endtime=end)
        inv = read_inventory(par.stationxmldir + "*.xml")
        stats.resample(sampling_rate=50)
        stats.merge(method=1,fill_value="latest")
        stats.attach_response(inv)
        stats.remove_sensitivity()
        stats.sort()
        stats.reverse()
        stats.trim(start,end)

        if stats.select(component="2"):
            stats.rotate(method='->ZNE', inventory=inv, components=['Z23'])
        else:
            stats.rotate(method='->ZNE', inventory=inv, components=['ZNE'])

        l_lon = inv.get_coordinates(stats[0].id,t1)["longitude"]
        l_lat = inv.get_coordinates(stats[0].id,t1)["latitude"]
        height = inv.get_coordinates(stats[0].id,t1)["elevation"] - inv.get_coordinates(stats[0].id,t1)["local_depth"]
        print(stats)
        fs = stats[0].stats.sampling_rate
        stats.detrend("linear")


        stats.detrend("simple")
        stats[0].filter('highpass',freq=par.freqmin)
        stats[1].filter('highpass',freq=par.freqmin)
        stats[2].filter('highpass',freq=par.freqmin)
        if first:
            first = False
            o_lon = l_lon
            o_lat = l_lat
            o_height = height

        lon,lat = util.util_geo_km(o_lon,o_lat,l_lon,l_lat)
        coo.append([lon*1000,lat*1000,height-o_height])


        tsz.append(stats[0].data)
        tsn.append(stats[1].data)
        tse.append(stats[2].data)

    ttse  =  np.array(tse)
    ttsn  =  np.array(tsn)
    ttsz  =  np.array(tsz)

    subarray = np.array(subarray)
    vp = 1000.
    vs = 560.
    sigmau = 0.0000001
    result = AA.array_rotation_strain(subarray, np.transpose(ttse), np.transpose(ttsn), np.transpose(ttsz), vp, vs, np.array(coo), sigmau)

    rotz = result['ts_w3']
    rotn = result['ts_w2']
    rote = result['ts_w1']

    straine = result['ts_e'][:,0,0]
    strainn = result['ts_e'][:,1,1]
    strainz = result['ts_e'][:,2,2]
    strainv = result['ts_d']
    
    _,Ref,_ = array_stations[subarray[0]].split('.')
    rots = stats.copy()
    rots[0].stats.station = Ref
    rots[0].stats.channel = "BJZ"
    rots[0].stats.location = "10"
    rots[0].data = rotz
    rots[1].stats.station = Ref
    rots[1].stats.channel = "BJN"
    rots[1].stats.location = "10"
    rots[1].data = rotn
    rots[2].stats.station = Ref
    rots[2].stats.channel = "BJE"
    rots[2].stats.location = "10"
    rots[2].data = rote
    rots.detrend("simple")

    # and the parameter periods_per_window
    rots.trim(start,end)
    rots.plot()

    myday = str(rots[0].stats.starttime.julday)

    pathyear = str(rots[0].stats.starttime.year)
    # open catalog file in read and write mode in case we are continuing d/l,
    # so we can append to the file
    mydatapath = os.path.join(par.path2store, pathyear)

    # create datapath 
    if not os.path.exists(mydatapath):
        os.mkdir(mydatapath)

    mydatapath = os.path.join(mydatapath, rots[0].stats.network)
    if not os.path.exists(mydatapath):
        os.mkdir(mydatapath)

    mydatapath = os.path.join(mydatapath, rots[0].stats.station)

    # create datapath 
    if not os.path.exists(mydatapath):
                os.mkdir(mydatapath)

    for i,tr in enumerate(rots):
        mydatapathchannel = os.path.join(mydatapath,rots[i].stats.channel + ".D")

        if not os.path.exists(mydatapathchannel):
            os.mkdir(mydatapathchannel)

        netFile = rots[i].stats.network + "." + rots[i].stats.station +  "." + rots[i].stats.location + "." + rots[i].stats.channel+ ".D." + pathyear + "." + myday
        netFileout = os.path.join(mydatapathchannel, netFile)

        # try to open File
        try:
            netFileout = open(netFileout, 'ab')
        except:
            netFileout = open(netFileout, 'w')
        # header of the stream object which contains the output of the ADR
        rots[i].write(netFileout , format='MSEED')
        netFileout.close()

    return end


['XM.KLT0.BH?','XM.KLT1.SH?','XM.KLT2.SH?','XM.KLT3.SH?']\
['XM.GRW0.BH?','XM.GRW2.SH?','XM.GRW3.SH?']\
['XM.PAS0.BH?','XM.PAS1.SH?','XM.PAS2.SH?']\
['XM.KEN0.BH?','XM.KEN1.SH?','XM.KEN2.SH?','XM.KEN3.SH?']

In [ ]:
path2store = "./data_adr"

#"1998-11-04T22:29" - "1998-11-04T22:40" 
#"2001-10-26T05:00" - "2001-10-26T05:20"
#"2001-10-19T14:20" - "2001-10-19T16:30"
#"2001-11-08T14:00" - "2001-11-08T14:30"

array=['XM.KLT0.BH?','XM.KLT1.SH?','XM.KLT2.SH?','XM.KLT3.SH?']
#array=['XM.GRW0.BH?','XM.GRW1.SH?','XM.GRW2.SH?','XM.GRW3.SH?']
#array=['XM.PAS0.BH?','XM.PAS1.SH?','XM.PAS2.SH?']
#array=['XM.KEN0.BH?','XM.KEN1.SH?','XM.KEN2.SH?','XM.KEN3.SH?']
sub_array = [0,1,2,3]
t1 = UTCDateTime("1998-11-04T22:29")
t2 = UTCDateTime("1998-11-04T22:40")

par = getParameters(path2store,t1,t2,array,sub_array)


try:
    run(par)
except:
    print("%s failed"%t1)

